# Scraping web data
* There are many different ways to scrape data off of the web, and it's useful to understand some of the mechanisms involved
* Often organizations don't want to enable data to be scraped, either to drive traffic to pay-for APIs, or to limit access to within the providers platform
* Some have gone as far as to suggest scraping data might be illegal, though the hiQ Labs v. LinkedIn case of last year challenged that perspective (an opinionated perspective from the EFF)
  * https://www.eff.org/deeplinks/2019/09/victory-ruling-hiq-v-linkedin-protects-scraping-public-data
* Regardless, it's certainly legal in some contexts, it's a bit of a cat and mouse game, and this is a great time to teach a little bit more about web scraping in the context of this assignment

# Restricting access
There are a few main ways a website might try and restrict access to web scrapers, short of requiring user logins or passwords.

1. __IP address blocking__. In this approach the website general maintains a list of IP address ranges which are not allowed to access the data (this list is often referred to as a blacklist or, more recently, a block list). The IP address is very difficult to fabricate (called spoofing) and still allow the scraper to get the data they are requesting back, so the main method of circumvention is to get access to a network which is allowed to retrieve data. This can be done through a proxy service to another network. This mechanism is very difficult to implement unless the web scrapers network information is well known, however it is __very common__ for a scraper who is making many fast requests to be "auto-blocked" based soely on IP address. The base lesson here is if you are scraping, limit the number of requests you make per minute.
2. __robots.txt__. In this approach the website maintains a file at the root of the website directory called robots.txt which outlines the rules which are expected for web scraping. For instance, the site `http://umich.edu/robots.txt` contains a single rule for all user-agents (all kinds of browsers) which is that they are not allowed to go to pages starting with `/admin/` or `/pingtest/`. These rules are completely optional and there is no mechanism for enforcement in this protocol, but many software libraries will automatically assume the user wants to respect the contents of the robots.txt file if it exists (and thus have mechanisms for turning off this feature). The University of Michigan robots.txt file:

```
User-agent: *
Disallow: /admin/
Disallow: /pingtest/
```

3. __User-agent headers__. When a browser makes an HTTP request the request and response are each made up of two pieces, the *headers* which are key/values pairs, and the *content*, which is the detail of the request/response. The request is quite often empty of content, and the response is often HTML or binary depending upon the file being requested. The user-agent is a long string which historically has been used to describe the capabilities of the browser but is actually made up of many different pieces which typically describe capabilities, operating system, plugins installed, etc. Most libraries which allow for web scraping allow you to override the default user-agent and supply your own, thus spoofing the software the request is coming from.
4. __Cookies__. Many websites will only work if cookies, which are small text strings, are stored on the requesting machine and resent with each request. Cookies are one of the key mechanisms websites use to determine a user "session", however it can be a useful mechanism for blocking requests as many scraping tools out-of-the-box are not configured to persist cookies between subsequent calls to a web service (however, most have the capacity).
5. __Javascript__. This approach is often an unintentional method of blocking scraping access and is a side effect of using web frameworks which are javascript heavy, the React framework being a good example of such. In this mechanism the website data is essentially a series of scripts which retrieve data dynamically from the server back end based on the activities of the user. What appears to be a single web request is actually many smaller requests based on events such as elements scrolling or rendering to the screen. The most common mechanism to circumvent this is to route all web scraping activity through a browser using a framework such as Selenium. This unfortunatly has a very high development load, and feels more akin to "reverse engineering" then "scraping", however it effectively circumvents issues with cookies, user-agent headers, and robots.txt at the same time, and provides ample opportunities for circumventing authentication (by using prestored credentials). It is much harder to detect than other mechanisms for web scraping.
6. __captcha__. With captcha's the website requires browsers to engage in a "human intelligence task" in order to load a webpage, making it difficult to scrape. This also makes the website a complete pain in the ass to use for humans, so it tends to be used only in high stakes activities (e.g. account creation).

# Gaining access
Each of the mechanisms above can be defeated, and this is important to be aware of when you are building and deploying a web service. Without a shared secret (e.g. password), preventing scraping entirely is intractable. How you approach each scraping issue depends on the mechanisms being used (above), as well as the resources at your disposal (time, networks, money, effort). As an example, consider the website https://bigten.org/calendar.aspx?path=football&year=2019 . This website uses a combination of approaches to limit the ability to scrape it. A good way of exploring this is to load the website into the Chrome browser, then open the Developer Tools in Chrome and select the network tab, then reload the website and click the first item. This will render the list of requests and response headers that the browser has made to the website.

<img src="big1.png" />

In this example, a number of mechanisms are being used, including the setting of cookies and the redirection of content. A simple way to work around the these methods is to copy the request headers shown at the bottom (and in file `headers.txt`. These headers outline various negotiations between the browser and the server, but most relevant are the user-agent string and the cookies. These headers can be supplied to web scraping programs to "spoof" the request. `curl` is one example of these programs, and can be well integrated with Jupyter notebooks (you may need to install this first by going to the console and typing `sudo apt-get update; sudo apt-get install curl`. One you have done this you can download the contents of the webpage using the below.

(Note, this command is saving the results into a file called `outfile.gz`, note that the andpersand in the URL needs to be escaped, and that I enabled verbose mode with `-v` just for educational purposes)

In [3]:
!curl -vH @headers.txt --output outfile.gz https://bigten.org/calendar.aspx?path=football\&year=2019

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0*   Trying 67.192.28.17...
* TCP_NODELAY set
* Connected to bigten.org (67.192.28.17) port 443 (#0)
* ALPN, offering h2
* ALPN, offering http/1.1
* successfully set certificate verify locations:
*   CAfile: /etc/ssl/certs/ca-certificates.crt
  CApath: /etc/ssl/certs
} [5 bytes data]
* TLSv1.3 (OUT), TLS handshake, Client hello (1):
} [512 bytes data]
* TLSv1.3 (IN), TLS handshake, Server hello (2):
{ [95 bytes data]
* TLSv1.2 (IN), TLS handshake, Certificate (11):
{ [3890 bytes data]
* TLSv1.2 (IN), TLS handshake, Server key exchange (12):
{ [333 bytes data]
* TLSv1.2 (IN), TLS handshake, Server finished (14):
{ [4 bytes data]
* TLSv1.2 (OUT), TLS handshake, Client key exchange (16):
} [70 bytes data]
* TLSv1.2 (OUT), TLS change cipher, Client hello (1):

The content file can now be opened and processed with your favorite HTML processing library.

In [12]:
import gzip
from bs4 import BeautifulSoup
import pandas as pd
with gzip.open("outfile.gz") as contents:
    #dfs=pd.read_html(contents)
    #print(dfs[0].head())
    soup = BeautifulSoup(contents.read(), 'lxml')
    print(soup.tr) #or whatever your other HTML processing is

<tr class="sidearm-calendar-table-thead-tr">
<th class="sidearm-calendar-list-group-list-game-team x-small-12 accessible-hidden-cell">Date</th>
<th class="sidearm-width-25 sidearm-calendar-list-group-list-game-team x-small-12">Away</th>
<th class="sidearm-width-25 sidearm-calendar-list-group-list-game-team sidearm-calendar-list-group-list-game-opponent">Home</th>
<th class="sidearm-width-10 sidearm-calendar-list-group-list-game-team sidearm-calendar-list-group-list-game-time" data-bind="text: (final ? 'Result' : 'Time')">Time</th>
<th class="sidearm-width-20 sidearm-calendar-list-group-list-game-team sidearm-calendar-list-group-list-game-location">Location</th>
<th class="sidearm-width-20 sidearm-calendar-list-group-list-game-team sidearm-calendar-list-group-list-game-media">Links</th>
</tr>


In [10]:
pd.read_html("outfile2")

[   Date    Away    Home                         Time       Location  \
 0   NaN  (Away)  (Home)  Final  Cancelled  Postponed  (Conf.)  (DH)   
 
                                           Links  
 0  Stats  Audio  Video  Box Score  Recap  Video  ]

In [ ]:
/html/body/form/main/article/div/section[1]/section[1]/div/section[1]/div/div/table/tbody/
#DataTables_Table_0 > tbody > tr:nth-child(1) > th